# Aquisição de Dados com Python

**Conteúdo:**
- O que não te contaram sobre análise de dados
- Web scraping
- BeautifulSoup
- Requests
- Páginas dinâmicas
- Extração de dados de arquivos de Word
- Extração de dados de arquivos PDF
- Selenium

## O que não te contaram sobre análise de dados
<img src="https://i.imgur.com/iX7RLQm.png">
<center>Banko, M., Brill E. (2001). <i>Scaling to Very Very Large Corpora for Natural Language Disambiguation.</i></center>
<br><br>
No mundo ideal, todos os dados estariam disponíveis a qualquer momento, no melhor formato possível, sem erros ou omissões. No mundo real, a qualidade dos dados é muito variada e boa parte do tempo do analista de dados é despendido na aquisição e tratamento dos dados.
<br><br>
Sobre a aquisição de dados, uma dificuldade comumente encontrada é que eles não estão disponíveis em um formato diretamente consumível por um programa de computador para tarefas de análise. É muito comum que os dados que se quer analisar estejam em formatos originalmente pensados apenas para um usuário humano visualizá-los, como uma página web ou um documento do Word, sem terem sido criadas Application User Interfaces (APIs) para que outros programas pudessem tratar aquela informação de forma automática.
<br><br>
Mas, se os dados estão disponíveis para usuários humanos através de algum aplicativo, seria possível criar um programa que simulasse o acesso de um usuário e coletasse "à força" aqueles dados?

## Web scraping
*Data scraping*\* (raspagem de dados) é o conjunto de técnicas de criação de códigos para coletar dados disponíveis em um formato pensado apenas no usuário final do sistema.
<br><br>
\*Não confundir scraping (do verbo *to scrape*, pronuncia-se "is<u>crei</u>pin") com scrapping (do verbo  *to scrap*, pronuncia-se "is<u>cré</u>pin")
<br><br>
*Web scraping* é o data scraping feito em páginas web. No Python, as bibliotecas **BeautifulSoup** e **Requests** são bastante utilizadas para web scraping.

## BeautifulSoup
A biblioteca *BeautifulSoup* facilita o acesso e a coleta de dados em arquivos HTML. 

In [ ]:
# Criando uma página HTML
html_exemplo = """
<html>
<head>
<title>Exemplo de página HTML</title>
</head>
<body>
<p id="paragrafo1">Olá!</p>
<p id="paragrafo2">Essa é uma página HTML. Clique <a href="https://html.spec.whatwg.org/">aqui</a> para saber mais.</p>
</body>
</html>
"""
print(html_exemplo)

In [ ]:
# Criando um soup a partir da página HTML
from bs4 import BeautifulSoup as bs
soup = bs(html_exemplo, "html.parser")
print(soup)

In [ ]:
# Embelezando o HTML
print(soup.prettify())

In [ ]:
# Acessando dos dados
soup.head

In [ ]:
soup.body

In [ ]:
# As propriedades do soup seguem a hierarquia dos elementos HTML
soup.html.head.title

In [ ]:
# Mas também é possível recuperar um elemento sem explicitar os elementos hierarquicamente superiores
soup.title

In [ ]:
# Caso haja mais de um elemento do mesmo tipo, apenas o primeiro é retornado
soup.p

In [ ]:
# soup.p é equivalente a soup.find("p")
soup.find("p")

In [ ]:
# Porém, o método find também permite outras formas de recuperar os elementos além do nome das tags
soup.find(id="paragrafo2")

In [ ]:
soup.find(attrs={"id": "paragrafo2"})

**Quiz 1** Como pegar o elemento "a"?

In [ ]:
# O método find sempre retorna apenas um elemento
# Para retornar uma lista de elementos que atendam a determinado critério, utilize o método find_all
soup.find_all("p")

In [ ]:
# A partir de um elemento, é possível recuperar o elemento pai e irmãos
p1 = soup.find("p")
p1.parent

In [ ]:
# No caso dos elementos irmãos, deve-se atentar para as quebras de linhas e demais strings, que são consideradas elementos
p1.next_sibling

In [ ]:
p1.next_sibling.next_sibling

In [ ]:
# next_sibling recupera os elementos irmãos a frente. Para pegar os elementos atrás, utilize previous_sibling
p2 = soup.find_all("p")[1]
p2.previous_sibling.previous_sibling

**Quiz 2** O que retorna `p2.previous_sibling`?

In [ ]:
# parent, next_sibling e previous_siblings consideram a hierarquia das tags no documento
# Para recuperar elementos desconsiderando a hierarquia, utilize next_element e previous_element
# As quebras de linhas e demais strings também são considerados elementos
p2.next_element

In [ ]:
p2.next_element.next_element

**Quiz 3** O que retorna `p2.next_element.next_element.next_element`?

In [ ]:
p1.previous_element

In [ ]:
p1.previous_element.previous_element

In [ ]:
# O texto interno de um elemento está armazenado na propriedade text
p1.text

In [ ]:
# Caso haja elementos filhos, a propriedade text concatena todos os textos internos
p2.text

In [ ]:
# É possível recuperar o valor de algum atributo de uma tag
link = soup.a
link["href"]

In [ ]:
# Para pegar todos os atributos e valors, user a propriedade attrs
link.attrs

## Requests
A biblioteca *Requests* permite emitir requisições HTTP.

In [ ]:
# Fazendo uma requisição
# 200 = requisição bem-sucedida
import requests
r_contabeis = requests.get("https://www.contabeis.com.br/tabelas/ufesp/")
r_contabeis

In [ ]:
# Resgatando o código HTML da página
print(r_contabeis.text)

In [ ]:
# Criando um soup
from bs4 import BeautifulSoup as bs
soup_contabeis = bs(r_contabeis.text, "html.parser")

In [ ]:
# Recuperando dados
# Utilize as ferramentas de desenvolvedor de um navegador para inspecionar a página
uls = soup_contabeis.find_all(class_="itemList")
ul_2019 = uls[0]
li_valor_2019 = ul_2019.find_all("li")[1]
print("O valor da UFESP de 2019 é " + li_valor_2019.text)

**Quiz 4** O que retorna `list(li_valor_2019.next_siblings)[3].text`?

## Páginas dinâmicas
Muitas páginas utilizam JavaScript para modificar o código HTML da página após ela ter sido renderizada pelo navegador.
A biblioteca Requests **não** executa JavaScript.

In [ ]:
r_sefaz = requests.get("https://portal.fazenda.sp.gov.br/Paginas/Indices.aspx")
print("UFESP" in r_contabeis.text)
print("UFESP" in r_sefaz.text)

Em páginas dinâmicas, uma alternativa para coletar os dados é utilizar as ferramentas de desenvolvedor de um navegador e analisar o funcionamento da página para identificar de onde vem a informação.

In [ ]:
# Após utilizarmos as ferramentas de desenvolvedor para inspecionar o tráfego de requisições
# foi possível identificar uma endereço que retorna um XML com as informações desejadas
r_indices_sefaz = requests.get(
    "https://portal.fazenda.sp.gov.br/_api/Web/Lists/getByTitle('Indices')/items?$Select=Title,Valor"
)
soup_indices = bs(r_indices_sefaz.text, "html.parser") # BeautifulSoup também trabalha com XML
print(soup_indices.prettify())

In [ ]:
d_valor_list = soup_indices.find_all("d:valor")
d_valor_2019 = d_valor_list[-1]
print("O valor da UFESP de 2019 é " + d_valor_2019.text)

## Extração de dados de arquivos de Word
Os arquivos de Word (\*.docx) possuem um estrutura em XML. Portanto, são passíveis de serem manipulados pela biblioteca BeautifulSoup.

In [ ]:
import zipfile
from bs4 import BeautifulSoup as bs

with zipfile.ZipFile("documento.docx", 'r') as zfp:
    with zfp.open('word/document.xml') as doc_xml:
        docx_soup = bs(doc_xml.read(), 'xml')

print(docx_soup.prettify())

In [ ]:
# Recuperando dados
docx_soup.find_all("w:t")

**Quiz 5** O que retorna `docx_soup.text`?

## Extração de dados de arquivos PDF
Arquivos PDF são menos estruturados para extração de texto e as informações coletadas estão sujeitas a um saneamento pois é comum surgirem caracteres estranhos em meio ao texto. Porém, a extração do texto "bruto" é uma tarefa fácil de ser realizada com a biblioteca *PyPDF2*.

**Atenção:** a biblioteca PyPDF2 não faz parte do conjunto de pacotes padrão do Anaconda e deve ser instalada.

In [ ]:
import PyPDF2

pdf_file = open('documento.pdf', 'rb')
pdf_reader = PyPDF2.PdfFileReader(pdf_file)
page = pdf_reader.getPage(0) # Contagem de páginas começa no 0
text = page.extractText()
print(repr(text))
pdf_file.close()

In [ ]:
# Usando expressões regulares para recuperar as etapas do procedimento de recuperação de senha indo pessoalmente
import re

re.split("Posto Fiscal", text)

In [ ]:
trecho = re.split("Posto Fiscal", text)[1]
trecho

In [ ]:
trecho = re.split("ATENÇÃO!", trecho)[0]
trecho

In [ ]:
etapas = re.split("\n\w\)", trecho)
etapas

In [ ]:
etapas = etapas[1:]
etapas

In [ ]:
etapas = [etapa.strip() for etapa in etapas]
etapas

## Selenium
O Selenium é uma biblioteca que permite a execução automática de comandos em um navegador. Originalmente criado para automatização de testes em páginas web, ele pode ser utilizado para realização de web scraping quando não for possível analisar o funcionamento de uma página dinâmica.


**Atenção:** a biblioteca Selenium não faz parte do conjunto de pacotes padrão do Anaconda e deve ser instalada.

In [ ]:
# Configurando o WebDriver do Chrome
from selenium import webdriver

driver = webdriver.Chrome("chromedriver.exe") # Baixe o Chrome WebDriver em https://chromedriver.chromium.org/

In [ ]:
# Acessando uma site e colentando os dados
import time

driver.get("https://portal.fazenda.sp.gov.br/Paginas/Indices.aspx")
time.sleep(1) # Aguarde o carregamento dos dados
td_valor_list = driver.find_elements_by_class_name("valor")
td_valor_2019 = td_valor_list[0]
print("O valor da UFESP de 2019 é " + td_valor_2019.text)

Apesar de conveniente, o uso do Selenium tem algumas desvantagens em relação a análise da página dinâmica para coleta de dados:

- Qualquer mudança na página pode impactar o script
- É necessário se preocupar com as versões do navegador e do webdriver
- Os scripts que utilizam Selenium serão mais lentos pois dependem do carregamento da página em um navegador